# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets. 

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide.   Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located on Kaggle (https://www.kaggle.com/kazanova/sentiment140). Once you have downloaded and imported the dataset, it you will need to define the columns names: df.columns = ['target','id','date','flag','user','text']

*Notes:* 

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [1]:
# your code here

import pandas as pd
import zipfile

zip_file_path = '/Users/belma/Documents/IRONHACK/Lessons/Week_8/Day_3/Morning/lab-nlp-8.05/archive.zip'
extract_path = 'lab-nlp-8.05'

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)


In [2]:
csv_file_path = 'lab-nlp-8.05/training.1600000.processed.noemoticon.csv'

df = pd.read_csv(csv_file_path, encoding='latin-1', header=None)
df.head()

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [3]:
# Define the column names as mentioned: ['target', 'id', 'date', 'flag', 'user', 'text'].

df.columns = ['target', 'id', 'date', 'flag', 'user', 'text']
df.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [4]:
sample_size = 20000  # Define the desired sample size
df = df.sample(n=sample_size, random_state=42)

### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](data-cleaning-results.png)

In [5]:
# your code here

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
import re

In [6]:
# download wordnet first, because i received error 
#nltk.download('wordnet')
#nltk.download('omw-1.4')
#nltk.download('stopwords')

In [7]:
'''def remove_stopwords(l):
    lemmatized = [wnl.lemmatize(word) for word in l]
    without_sw = [word for word in lemmatized if word not in stopwords.words()]
    return without_sw

df['text_processed'] = df['text'].apply(clean_up)
df['text_processed'] = df['text_processed'].apply(tokenize)

ps = PorterStemmer()
wnl = WordNetLemmatizer()

def stem_and_lemmatize(l):
    stemmed = [ps.stem(w) for w in l]
    lemmatized = [wnl.lemmatize(word) for word in stemmed]
    return lemmatized

df['text_processed'] = df['text_processed'].apply(stem_and_lemmatize)
df['text_processed'] = df['text_processed'].apply(remove_stopwords)'''

"def remove_stopwords(l):\n    lemmatized = [wnl.lemmatize(word) for word in l]\n    without_sw = [word for word in lemmatized if word not in stopwords.words()]\n    return without_sw\n\ndf['text_processed'] = df['text'].apply(clean_up)\ndf['text_processed'] = df['text_processed'].apply(tokenize)\n\nps = PorterStemmer()\nwnl = WordNetLemmatizer()\n\ndef stem_and_lemmatize(l):\n    stemmed = [ps.stem(w) for w in l]\n    lemmatized = [wnl.lemmatize(word) for word in stemmed]\n    return lemmatized\n\ndf['text_processed'] = df['text_processed'].apply(stem_and_lemmatize)\ndf['text_processed'] = df['text_processed'].apply(remove_stopwords)"

In [8]:
'''df['text_processed'] = df['text'].apply(clean_up)
df['text_processed'] = df['text_processed'].apply(tokenize)
df['text_processed'] = df['text_processed'].apply(stem_and_lemmatize)
df['text_processed'] = df['text_processed'].apply(remove_stopwords)'''

"df['text_processed'] = df['text'].apply(clean_up)\ndf['text_processed'] = df['text_processed'].apply(tokenize)\ndf['text_processed'] = df['text_processed'].apply(stem_and_lemmatize)\ndf['text_processed'] = df['text_processed'].apply(remove_stopwords)"

In [9]:
'''# Function to clean up the text data
def clean_up(s):
    s = re.sub(r'\d+', ' ',s)
    s = re.sub(r'http\S+', ' ',s)
    s = re.sub(r'[^\w\s]', ' ',s)
    s = s.lower()
    return s

# Function to tokenize the text data
def tokenize(s):
    s = word_tokenize(s)
    return s

# Function to stem and lemmatize the tokens
def stem_and_lemmatize(l):
    ps = PorterStemmer()
    stemmed = [ps.stem(w) for w in l]
    
    lemmatizer = WordNetLemmatizer() 
    lemmatized = [lemmatizer.lemmatize(word) for word in stemmed]
    return lemmatized

# Function to remove stopwords from the tokens
def remove_stopwords(l):
    without_sw = [word for word in lemmatized if not word in stopwords.words()]
    return without_sw'''

"# Function to clean up the text data\ndef clean_up(s):\n    s = re.sub(r'\\d+', ' ',s)\n    s = re.sub(r'http\\S+', ' ',s)\n    s = re.sub(r'[^\\w\\s]', ' ',s)\n    s = s.lower()\n    return s\n\n# Function to tokenize the text data\ndef tokenize(s):\n    s = word_tokenize(s)\n    return s\n\n# Function to stem and lemmatize the tokens\ndef stem_and_lemmatize(l):\n    ps = PorterStemmer()\n    stemmed = [ps.stem(w) for w in l]\n    \n    lemmatizer = WordNetLemmatizer() \n    lemmatized = [lemmatizer.lemmatize(word) for word in stemmed]\n    return lemmatized\n\n# Function to remove stopwords from the tokens\ndef remove_stopwords(l):\n    without_sw = [word for word in lemmatized if not word in stopwords.words()]\n    return without_sw"

In [10]:
#I've rearranged the code to define the wnl (WordNetLemmatizer) variable before the remove_stopwords function. 
#This ensures that the lemmatized variable is accessible within the remove_stopwords function.


# Function to clean up the text data
def clean_up(s):
    s = re.sub(r'\d+', ' ',s)
    s = re.sub(r'http\S+', ' ',s)
    s = re.sub(r'[^\w\s]', ' ',s)
    s = s.lower()
    return s

# Function to tokenize the text data
def tokenize(s):
    s = word_tokenize(s)
    return s

# Function to remove stopwords from the tokens
def remove_stopwords(l):
    lemmatized = [wnl.lemmatize(word) for word in l]
    without_sw = [word for word in lemmatized if word not in stopwords.words()]
    return without_sw

df['text_processed'] = df['text'].apply(clean_up)
df['text_processed'] = df['text_processed'].apply(tokenize)

ps = PorterStemmer()
wnl = WordNetLemmatizer()

# Function to stem and lemmatize the tokens
def stem_and_lemmatize(l):
    stemmed = [ps.stem(w) for w in l]
    lemmatized = [wnl.lemmatize(word) for word in stemmed]
    return lemmatized

df['text_processed'] = df['text_processed'].apply(stem_and_lemmatize)
df['text_processed'] = df['text_processed'].apply(remove_stopwords)

In [11]:
df.head()

,target,id,date,flag,user,text,text_processed
541200,0,2200003196,Tue Jun 16 18:18:12 PDT 2009,NO_QUERY,LaLaLindsey0609,@chrishasboobs AHHH I HOPE YOUR OK!!!,"[chrishasboob, ahhh, hope]"
750,0,1467998485,Mon Apr 06 23:11:14 PDT 2009,NO_QUERY,sexygrneyes,"@misstoriblack cool , i have no tweet apps fo...","[misstoriblack, cool, tweet, app, razr]"
766711,0,2300048954,Tue Jun 23 13:40:11 PDT 2009,NO_QUERY,sammydearr,@TiannaChaos i know just family drama. its la...,"[tiannachao, famili, drama, lame, hey, time, h..."
285055,0,1993474027,Mon Jun 01 10:26:07 PDT 2009,NO_QUERY,Lamb_Leanne,School email won't open and I have geography ...,"[school, email, open, geographi, stuff, revis,..."
705995,0,2256550904,Sat Jun 20 12:56:51 PDT 2009,NO_QUERY,yogicerdito,upper airways problem,"[upper, airway, problem]"


### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [12]:
# your code here
# Import the necessary libraries
from nltk import FreqDist
from nltk import NaiveBayesClassifier

# Combine all the words in the text_processed column
all_words = [word for sublist in df['text_processed'] for word in sublist]

all_words

['chrishasboob',
 'ahhh',
 'hope',
 'misstoriblack',
 'cool',
 'tweet',
 'app',
 'razr',
 'tiannachao',
 'famili',
 'drama',
 'lame',
 'hey',
 'time',
 'hang',
 'guy',
 'sleepov',
 'whatev',
 'call',
 'school',
 'email',
 'open',
 'geographi',
 'stuff',
 'revis',
 'stupid',
 'school',
 'upper',
 'airway',
 'problem',
 'miss',
 'pastor',
 'sermon',
 'faith',
 'lunch',
 'dj',
 'eat',
 'piginthepok',
 'whi',
 'feel',
 'gahh',
 'noo',
 'peyton',
 'live',
 'horribl',
 'mrstessyman',
 'glad',
 'product',
 'review',
 'bit',
 'site',
 'enjoy',
 'knit',
 'perezhilton',
 'zach',
 'make',
 'pee',
 'grown',
 'gay',
 'sum',
 'day',
 'word',
 'kacker',
 'wkj',
 'great',
 'mind',
 'alik',
 'poorli',
 'bed',
 'lilpecan',
 'realli',
 'great',
 'small',
 'blizzard',
 'cold',
 'wind',
 'blow',
 'wize',
 'calm',
 'day',
 'loan',
 'offer',
 'onli',
 'feel',
 'quit',
 'sleepi',
 'today',
 'wish',
 'stay',
 'bed',
 'today',
 'year',
 'school',
 'nadalnew',
 'mathieu',
 'total',
 'choke',
 'rd',
 'set',
 'rog

In [13]:
# Calculate the frequency distribution of all words
freq_dist = FreqDist(all_words)

freq_dist

FreqDist({'wa': 1363, 'day': 1328, 'work': 1120, 'love': 1030, 'quot': 984, 'today': 870, 'time': 834, 'miss': 695, 'back': 678, 'feel': 649, ...})

In [14]:
#Select the top 5,000 words from the frequency distribution
top_words = freq_dist.most_common(5000)

top_words

[('wa', 1363),
 ('day', 1328),
 ('work', 1120),
 ('love', 1030),
 ('quot', 984),
 ('today', 870),
 ('time', 834),
 ('miss', 695),
 ('back', 678),
 ('feel', 649),
 ('realli', 644),
 ('amp', 619),
 ('hope', 597),
 ('watch', 593),
 ('night', 572),
 ('make', 545),
 ('home', 515),
 ('twitter', 448),
 ('great', 433),
 ('morn', 426),
 ('wish', 418),
 ('sleep', 415),
 ('wait', 414),
 ('follow', 404),
 ('tomorrow', 394),
 ('haha', 382),
 ('sad', 367),
 ('week', 364),
 ('sorri', 364),
 ('onli', 357),
 ('happi', 355),
 ('veri', 351),
 ('whi', 343),
 ('fun', 334),
 ('tonight', 323),
 ('friend', 315),
 ('nice', 313),
 ('start', 296),
 ('bed', 295),
 ('hate', 290),
 ('gon', 282),
 ('peopl', 281),
 ('tweet', 279),
 ('school', 272),
 ('show', 265),
 ('lt', 257),
 ('awesom', 255),
 ('final', 252),
 ('play', 251),
 ('hour', 248),
 ('everyon', 241),
 ('guy', 240),
 ('weekend', 231),
 ('year', 229),
 ('littl', 228),
 ('plea', 225),
 ('x', 220),
 ('long', 219),
 ('girl', 214),
 ('wan', 214),
 ('rain', 213)

In [15]:
'''# complete code snippet for this step
from nltk import FreqDist

# Combine all the words in the text_processed column
all_words = [word for sublist in df['text_processed'] for word in sublist]

# Calculate the frequency distribution of all words
freq_dist = FreqDist(all_words)

# Select the top 5,000 words from the frequency distribution
top_words = freq_dist.most_common(5000)'''

"# complete code snippet for this step\nfrom nltk import FreqDist\n\n# Combine all the words in the text_processed column\nall_words = [word for sublist in df['text_processed'] for word in sublist]\n\n# Calculate the frequency distribution of all words\nfreq_dist = FreqDist(all_words)\n\n# Select the top 5,000 words from the frequency distribution\ntop_words = freq_dist.most_common(5000)"

### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

[![Building Features](building-features.jpg)](https://www.youtube.com/watch?v=-vVskDsHcVc)

In [16]:
df.columns

Index(['target', 'id', 'date', 'flag', 'user', 'text', 'text_processed'], dtype='object')

In [17]:
# Create a list of tuples containing the bag-of-words features and the target output
features = [
    (
        {word: (word in tweet) for word in top_words},
        (sentiment == 4)
    )
    for tweet, sentiment in zip(df['text_processed'], df['target'])
]

# Print an example of the first feature set
print(features[0])

({('wa', 1363): False, ('day', 1328): False, ('work', 1120): False, ('love', 1030): False, ('quot', 984): False, ('today', 870): False, ('time', 834): False, ('miss', 695): False, ('back', 678): False, ('feel', 649): False, ('realli', 644): False, ('amp', 619): False, ('hope', 597): False, ('watch', 593): False, ('night', 572): False, ('make', 545): False, ('home', 515): False, ('twitter', 448): False, ('great', 433): False, ('morn', 426): False, ('wish', 418): False, ('sleep', 415): False, ('wait', 414): False, ('follow', 404): False, ('tomorrow', 394): False, ('haha', 382): False, ('sad', 367): False, ('week', 364): False, ('sorri', 364): False, ('onli', 357): False, ('happi', 355): False, ('veri', 351): False, ('whi', 343): False, ('fun', 334): False, ('tonight', 323): False, ('friend', 315): False, ('nice', 313): False, ('start', 296): False, ('bed', 295): False, ('hate', 290): False, ('gon', 282): False, ('peopl', 281): False, ('tweet', 279): False, ('school', 272): False, ('show'

### Building and Traininng Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

[![Building and Training NB](nb-model-building.jpg)](https://www.youtube.com/watch?v=rISOsUaTrO4)

In [18]:
# your code here
import random
from nltk import NaiveBayesClassifier
from nltk.classify import accuracy

# Split the feature set into a training set and a test set
train_size = int(0.8 * len(features))
train_set = features[:train_size]
test_set = features[train_size:]

# Create a Naive Bayes classifier and train it with the training set
classifier = NaiveBayesClassifier.train(train_set)

# Print the most informative features
classifier.show_most_informative_features()

# Evaluate the accuracy of the classifier on the test set
accuracy = accuracy(classifier, test_set)
print("Accuracy:", accuracy)

Most Informative Features
               ('_', 22) = False           False : True   =      1.0 : 1.0
            ('_anne', 3) = False           False : True   =      1.0 : 1.0
               ('aa', 5) = False           False : True   =      1.0 : 1.0
             ('aaah', 5) = False           False : True   =      1.0 : 1.0
              ('aah', 3) = False           False : True   =      1.0 : 1.0
             ('aahh', 4) = False           False : True   =      1.0 : 1.0
            ('aaron', 4) = False           False : True   =      1.0 : 1.0
          ('abandon', 5) = False           False : True   =      1.0 : 1.0
             ('abba', 3) = False           False : True   =      1.0 : 1.0
             ('abbi', 4) = False           False : True   =      1.0 : 1.0
Accuracy: 0.51


### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [19]:
# your code here
from nltk.classify import accuracy

# Test the classifier on the test set and calculate the accuracy score
accuracy_score = accuracy(classifier, test_set)

# Print the accuracy score
print("Accuracy:", accuracy_score)

Accuracy: 0.51


## Bonus Question 1: Improve Model Performance

If you are still not exhausted so far and want to dig deeper, try to improve your classifier performance. There are many aspects you can dig into, for example:

* Improve stemming and lemmatization. Inspect your bag of words and the most important features. Are there any words you should furuther remove from analysis? You can append these words to further remove to the stop words list.

* Remember we only used the top 5,000 features to build model? Try using different numbers of top features. The bottom line is to use as few features as you can without compromising your model performance. The fewer features you select into your model, the faster your model is trained. Then you can use a larger sample size to improve your model accuracy score.

In [20]:
# your code here
# Update the number of top features to be used
top_features_count = 5000

# Select the top features based on their frequency
top_words = [word for word, _ in freq_dist.most_common(top_features_count)]

# Create the feature set with the updated number of top features
features = [
    (
        {word: (word in tweet) for word in top_words},
        (sentiment == 4)
    )
    for tweet, sentiment in zip(df['text_processed'], df['target'])
]

# Split the feature set into a training set and a test set
train_size = int(0.8 * len(features))
train_set = features[:train_size]
test_set = features[train_size:]

# Train the Naive Bayes classifier with the updated feature set
classifier = NaiveBayesClassifier.train(train_set)

# Evaluate the accuracy of the classifier on the test set
accuracy_score = accuracy(classifier, test_set)

# Print the accuracy score
print("Accuracy:", accuracy_score)

Accuracy: 0.70525


## Bonus Question 2: Machine Learning Pipeline

In a new Jupyter Notebook, combine all your codes into a function (or a class). Your new function will execute the complete machine learning pipeline job by receiving the dataset location and output the classifier. This will allow you to use your function to predict the sentiment of any tweet in real time. 

In [21]:
# your code here

## Bonus Question 3: Apache Spark

If you have completed the Apache Spark advanced topic lab, what you can do is to migrate your pipeline from local to a Databricks Notebook. Share your notebook with your instructor and classmates to show off your achievements!

In [22]:
# your code here